ETL (pair 11)

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest

In [7]:
def descarga_csv(archivo):

    pd.read_csv(archivo, index_col=0)
    

In [8]:
df_cliente = descarga_csv('../files/clientes.csv')

In [9]:
df_cliente.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [50]:
df_clientes = pd.read_csv("../files/clientes.csv", index_col=0)

In [51]:
df_ventas = pd.read_csv("../files/ventas.csv", index_col=0)

In [52]:
# df = pd.read_csv("../files/productos.csv", engine="python")

df_productos = pd.read_csv("../files/productos.csv", delimiter=",", index_col=0)

In [57]:
def cambiar_a_unk(df,columnas):

    for c in columnas:
        try:
            df[c]= df[c].fillna('unknown')
        except:
            df[c]= df[c]

In [58]:
cambiar_a_unk(df_clientes,df_clientes.columns)

In [59]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 1 to 1000
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   first_name  1000 non-null   object
 1   last_name   1000 non-null   object
 2   email       1000 non-null   object
 3   gender      1000 non-null   object
 4   City        1000 non-null   object
 5   Country     1000 non-null   object
 6   Address     1000 non-null   object
dtypes: object(7)
memory usage: 62.5+ KB
